In [1]:
!pip install -q langchain
!pip install -qU transformers accelerate einops xformers bitsandbytes
!pip install -q unstructured
!pip install -q sentence_transformers
!pip install -q chromadb
!pip install langchain_community -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.1.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2024.1.0 which is incompatible.
dask-cuda 23.8.0 require

In [13]:
import pandas as pd
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from langchain.llms import HuggingFacePipeline

from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredExcelLoader


from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings


In [9]:
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token ('your_hf_token')"

In [3]:
import re
import glob

books_path = "/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/*.xlsx"
booklets = glob.glob(books_path)

def remove_unwanted(text):
    pattern = re.compile(r'\\ufdd0|[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+', flags=re.UNICODE)
    clean_text = pattern.sub('', text)
    
    return clean_text


In [4]:
for booklet in booklets:
    print(booklet)
    df= pd.read_excel(booklet, names=['row', 'text'])
    df['text'] = df['text'].apply(lambda x: remove_unwanted(str(x)))
    df.to_excel(booklet.split("/")[-1], index=False)

/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 2 Sections 1,2,3_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 1 Introduction Module Booklet 1TG_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 3 Section 4,5,6 ,7_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 5 Section 10_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 6_Section 11_final_04112021.xlsx
/kaggle/input/malawi-data/MW_TGBookletsExcel/MWTGBookletsExcel/TG Booklet 4 Sections 8, 9_final_04112021.xlsx


In [5]:
cleaned_booklets = glob.glob("/kaggle/working/*.xlsx")
loaders = [UnstructuredExcelLoader(f"{booklet}", engine="") for booklet in cleaned_booklets]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                     chunk_overlap=20)
chunked_documents = text_splitter.split_documents(docs)


In [7]:
model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [11]:
# Load model configuration from the pretrained model
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)

# Load the pretrained model for causal language modeling
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,                  # Identifier for the pretrained model
    trust_remote_code=True,    # Specify whether to trust remote code
    config=model_config,       # Model configuration loaded above
    quantization_config=bnb_config, # Configuration for quantization
    device_map='auto',         # Specify device mapping, in this case 'auto'
)

# Load the tokenizer associated with the pretrained model
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [14]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
# Create a Chroma object and initialize it with the provided documents, embeddings,
# and persist_directory to store the database.
vectordb = Chroma.from_documents(documents=chunked_documents, embedding=embeddings, persist_directory="chroma_db")

In [17]:
### huggingface pipeline
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto")

llm = HuggingFacePipeline(pipeline=query_pipeline)

2024-02-15 11:08:39.626508: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 11:08:39.626603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 11:08:39.751617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [19]:
retriever = vectordb.as_retriever(k=3)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True,
)


In [20]:
import time
def test_rag(qa, query):
    #print(f"Query: {query}\n")
    time_1 = time.time()
    result = qa.run(query)
    time_2 = time.time()
    time_taken = round(time_2-time_1, 3)

    return result,time_taken

In [21]:
test = pd.read_csv("/kaggle/input/malawi-data/Test.csv")

In [22]:
from tqdm import tqdm

results = []
sources = []
for question in tqdm(test["Question Text"]):
    
    try:
        result,_ = test_rag(qa, question)
        docs = vectordb.similarity_search(result)
        source = docs[0].metadata['source'].split("/")[-1]

        results.append(result)
        sources.append(source)
    except:

        results.append("Error")
        sources.append("Error")



/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...


In [23]:
result

" The national level should maintain contact with the district for daily updates through a designated communication channel, such as a dedicated phone line or email address. This is important because it ensures that the national level receives timely and accurate information about the progress of the epidemic at the district level, which can inform the national level's response and decision-making. Additionally, it helps to ensure that the district level is aware of the national level's response and any changes to the response plan."